# Wafer map pattern classification using MultiNN

- Directory

/data/WMPC_CNN_0_0_softmax.pickle
...
/data/WMPC_MFE_0_0_softmax.pickle
...

In [4]:
import pickle
import os
import sys

import numpy as np
from tensorflow.keras.layers import Input, Dense, MaxPooling2D, Concatenate
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
DIM = 64
REPLICATION = 10
BATCH_SIZE = 32
MAX_EPOCH = 1000
TRAIN_SIZE_LIST = [500, 5000, 50000, 162946]

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True, verbose=0)

In [14]:
with open('../data/y.pickle', 'rb') as f:
    y = pickle.load(f)
y_onehot = tf.keras.utils.to_categorical(y)

In [ ]:
def FNN(lr=1e-4):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(9, activation='softmax')])
    model.compile(optimizer= tf.keras.optimizers.Adam(lr=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
REP_ID = 0
RAN_NUM = 27407 + REP_ID
print('Replication:', REP_ID)
for TRAIN_SIZE_ID in range(4):
    TRAIN_SIZE = TRAIN_SIZE_LIST[TRAIN_SIZE_ID]

    y_trnval, y_tst =  train_test_split(y_onehot, test_size=10000, random_state=RAN_NUM)
    if TRAIN_SIZE == 162946:
            pass
    else:    
        y_trnval, _ = train_test_split(y_trnval, train_size=TRAIN_SIZE, random_state=RAN_NUM)

    filename_MFE = '../data/WMPC_'+'MFE_'+str(TRAIN_SIZE)+'_'+str(REP_ID)+'_'
    filename_CNN = '../data/WMPC_'+'CNN_'+str(TRAIN_SIZE)+'_'+str(REP_ID)+'_'

    with open(filename_MFE + 'softmax.pickle', 'rb') as f:
        y_trnval_hat_mfe, y_tst_hat_mfe = pickle.load(f)
    with open(filename_CNN + 'softmax.pickle', 'rb') as f:
        y_trnval_hat_cnn, y_tst_hat_cnn = pickle.load(f)
    X_trnval_concat = np.concatenate([y_trnval_hat_mfe, y_trnval_hat_cnn], axis=1)
    X_tst_concat = np.concatenate([y_tst_hat_mfe, y_tst_hat_cnn], axis=1)

    labels = np.unique(np.argmax(y_trnval, 1))

    
    model = FNN()
    log = model.fit(X_trnval_concat, y_trnval, validation_split=0.2, 
                    epochs=MAX_EPOCH, batch_size=BATCH_SIZE,
                    callbacks=[early_stopping], verbose=0)

    y_trnval_hat = model.predict(X_trnval_concat)  
    y_tst_hat = model.predict(X_tst_concat)
    macro = f1_score(np.argmax(y_tst, 1), np.argmax(y_tst_hat, 1), labels=labels, average='macro')
    micro = f1_score(np.argmax(y_tst, 1), np.argmax(y_tst_hat, 1), labels=labels, average='micro')
    cm = confusion_matrix(np.argmax(y_tst, 1), np.argmax(y_tst_hat, 1))

    filename = '../result/WMPC_'+'Stacking_'+str(TRAIN_SIZE)+'_'+str(REP_ID)+'_'
    with open(filename+'f1_score.pickle', 'wb') as f:
        pickle.dump([macro, micro, cm], f)
    with open(filename+'softmax.pickle', 'wb') as f:
        pickle.dump([y_trnval_hat,y_trnval], f)
    with open(filename+'coef_.pickle', 'wb') as f:
        pickle.dump(model.coef_, f)